In [ ]:
# Copyright © 2021 LEADERG Inc. All rights reserved. Please keep it private. Publish to internet is not allowed. 

In [ ]:
# Import necessary packages 
import sys
import pandas as pd
import time
import os

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
def intro():
	#print('The program is about to fetch data from Taiwan Stock Exchange.', flush = True)
    print('\n程式即將從台灣證券交易所獲取股票相關數據。')
	#print('Due to TWSE\'s regulation, the program has to fetch the information', flush = True)
    #print('slowly to prevent your IP address from a temporary ban.', flush = True)
    print('根據台灣證券交易所規定的關係，程式必須緩慢獲取數據，')
    print('以防止您的IP地址被暫時禁止訪問台灣證券交易所。')
	#print('We apologize in advance for your incovience. ', flush = True)
    print('對於您的不便，我們深表歉意。')
	#print('Please wait patiently while the program fetchs data, thank you!', flush = True)
    print('在程序下載數據時，請耐心等待，謝謝！\n')
	
intro()

In [ ]:
# parameters 
stockNum = pd.to_numeric(2330)
fromYear = pd.to_numeric(2017)
fromMonth = pd.to_numeric(1)
toYear = pd.to_numeric(2020)
toMonth = pd.to_numeric(5)
outputPath = "data/stock/stock-data.csv"

In [ ]:
# This step is used to check if a valid number set has been offer as well as producing the 
# number of iteration that is required to complete the fetching process
def iteration():
	try:
		c1, c2, num = 0, 0, 0
		c1 = toYear - fromYear
		c2 = toMonth - fromMonth
		num = c1*12+c2
		if num <= 0:
			print('請輸入有效的日期區間')
		elif (num%1)!=0: 
			print('請輸入有效的日期區間') 
		else:
			return num
	except:
		print('請輸入有效的日期區間')

In [ ]:
# I change the date into a 8 digits number. The plus 100 is implemented because when we say 
# from January to May we meant 5 month worth of data, but to the computer it means 5 - 1 which
# is 4. The plus 1 is implemented because TWSE's format requires a specific day and 00 won't work
num = toYear*10000+toMonth*100+1+100
temp = 0
data = pd.DataFrame()
for i in range(iteration()+1):
	try:
		num -= 100
		if (num%10000)<100:
			num -= 8800
		temp = pd.read_csv('http://www.twse.com.tw/en/exchangeReport/STOCK_DAY?response=csv&date={}&stockNo={}'.format(num, stockNum), header=1, usecols=[0,1,3,4,5,6])
		print('已成功獲取數據：', num//10000, num%10000//100, flush = True)
		data = pd.concat([temp[1:-4], data]) # The first row and the last four are text
		time.sleep(11)
	except:
		print('獲取數據時出現錯誤：', num, flush = True)
		pass

In [ ]:
# Although we only use close, but in the future we can possibly make use of the other information 
# to determine the close price, so I kept all the data		
data.columns=['date', 'volume', 'open', 'high', 'low', 'close']

In [ ]:
# This step is to check if any text slip through the process.
data = data.reset_index(drop=True)
split = data.date.str.split('/')
for i in range(len(data)):
	if(len(split[i])!=3):
		data = data.drop(i)

In [ ]:
# Kelvin: this step is to drop the data with volume = 0
dataNoNa = data[data.volume != '0']

In [ ]:
dataNoNa.to_csv(outputPath, index=False)